In [ ]:
from lusidtools.jupyter_tools import toggle_code

"""Structured Results Store for storage of Portfolio data

Attributes
----------
structured_results_store
virtual_document
luminesce
"""

toggle_code("Toggle Docstring")

# Structured Results Store Example

This notebook demostrates loading of a custom Portfolio dataset into the LUSID Structured Results Store.

Once loaded, the dataset can be retrieved as one document, or the individual fields accessed

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timezone
import io
import json
import pytz
from IPython.core.display import HTML

# Then import the key modules from the LUSID package (i.e. The LUSID SDK)
import lusid as lu
import lusid.api as la
import lusid.models as lm

# And use absolute imports to import key functions from Lusid-Python-Tools and other helper package

from lusid.utilities import ApiClientFactory
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.jupyter_tools import StopExecution
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
)

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = ApiClientFactory(
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

api_status = pd.DataFrame(
    api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict()
)

display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.9366.0,0.5.2880,"{'relation': 'RequestLogs', 'href': 'http://st..."


In [ ]:
srs_api = api_factory.build(lu.StructuredResultDataApi)

In [ ]:
scope = "srs"
effective_date = datetime(2021, 1, 27, tzinfo=timezone.utc)
sample_data_result_type = "UnitResult/Custom"

In [1]:
%load_ext nb_black

ModuleNotFoundError: No module named 'nb_black'

## Define functions to create the Data Map

In [2]:
def gen_srs_data_map(df, unique_columns, data_types):
    df_columns = [column for column in df.columns if column not in unique_columns]
    
    unique_columns = [lm.DataDefinition(address=f"UnitResult/{scope}/{unique_column}", name=unique_column, data_type=data_types[unique_column], key_type="PartOfUnique") 
                      for unique_column in unique_columns]
    
    srs_data_definitions = [lm.DataDefinition(address=f"UnitResult/{scope}/{column}", name=column, data_type=data_types[column], key_type="leaf") 
                            for column in df_columns]
    
    return lm.DataMapping(data_definitions=unique_columns + srs_data_definitions)

def create_data_map(df, code, version, unique_columns, data_types):
    srs_data_map = gen_srs_data_map(df,unique_columns, data_types)
    srs_data_map_key = lm.DataMapKey(version=version, code=code)
    
    try:    
        srs_api.create_data_map(
            scope=scope, 
            request_body={
                code: lm.CreateDataMapRequest(
                    id = srs_data_map_key,
                    data = srs_data_map
                )
            }
        )
    except lu.ApiException as e:
        detail = json.loads(e.body)
        if detail['code'] not in [461]:
            raise e
    return srs_data_map_key

## Read in sample data

In [3]:
df = pd.read_csv("data/srs_custom_data.csv")
df = df.round(0)

## Upsert data into the Structured Results Store

In [4]:
def load_data():
    srs_ids=[]
    
    version = "1.01"
    portfolios = df.groupby("Port")
    
    sample_data_map_key = create_data_map(
        df = df,
        code = 'sample_data_map',
        version = "0.0009",
        unique_columns = ["Currency", "Date", "Sys", "Class"],
        data_types = { "RowId": "string", 
                       "Port": "string",
                       "Currency": "string",
                       "Date": "string",
                       "Sys": "string",
                       "Value": "decimal",
                       "Class": "string"}
    )

    for portfolio_id, pf_df in portfolios:    
        srs_id = lm.StructuredResultDataId(
            source = "Client", 
            code = portfolio_id, 
            effective_at = effective_date,
            result_type = sample_data_result_type)
    
        srs_ids.append(srs_id) 
    
        csv_data = io.StringIO()
        pf_df.to_csv(csv_data)   
        
        request_body = {
            portfolio_id: lm.UpsertStructuredResultDataRequest(
                id = srs_id,
                data = lm.StructuredResultData(
                    document_format = "csv",
                    version = version,
                    name = "Data file",
                    document = csv_data.getvalue(),
                    data_map_key = sample_data_map_key
                )
            )
        }
        
        result = srs_api.upsert_structured_result_data(
            scope=scope,
            request_body=request_body)

        display(pd.DataFrame(result.values.items()))
        
    return srs_ids

srs_ids = load_data()

,0,1
0,PortA,2022-06-01 19:08:44.160550+00:00


,0,1
0,PortB,2022-06-01 19:08:44.672188+00:00


,0,1
0,PortC,2022-06-01 19:08:45.270096+00:00


## Retrieve the raw data from the Structured Results Store

In [5]:
def retrieve_data():
    portfolio_ids = df['Port'].unique()

    for portfolio_id in portfolio_ids:
        srs_id = lm.StructuredResultDataId(
            source = "Client", 
            code = portfolio_id, 
            effective_at = effective_date,
            result_type = sample_data_result_type)
        
        result = srs_api.get_structured_result_data(
            scope = scope, 
            request_body = {
                "key": srs_id
            }
        )
    
        csv_data = io.StringIO(result.values["key"].document)
        doc = pd.read_csv(csv_data)
        display(srs_id.code)
        display(doc)
        
retrieve_data()

'PortA'

,Unnamed: 0,Port,Currency,Date,Sys,Value,Class
0,0,PortA,AUD,20210713,FSC,50000000,C
1,1,PortA,CAD,20210713,FSC,138234286,C
2,2,PortA,CHF,20210713,FSC,203402,C
3,3,PortA,EUR,20210713,FSC,235402950,C
4,4,PortA,EUR,20210715,FTC,22342054,C
5,5,PortA,EUR,20210727,FTC,2349130942,C
6,6,PortA,EUR,20210812,FTC,23489234,C
7,7,PortA,GBP,20210713,FSC,83425656,C
8,8,PortA,GBP,20210812,FTC,54354564,C
9,9,PortA,JPY,20210713,FSC,654659,C


'PortB'

,Unnamed: 0,Port,Currency,Date,Sys,Value,Class
0,26,PortB,AUD,20210713,FSC,5590075,B
1,27,PortB,AUD,20210812,FTC,5453435,B
2,28,PortB,SGD,20210713,FSC,43545554,B
3,29,PortB,USD,20210713,FSC,543543435453,B
4,30,PortB,EUR,20210713,FSC,54545435,B


'PortC'

,Unnamed: 0,Port,Currency,Date,Sys,Value,Class
0,31,PortC,AUD,20210713,FSC,75174369,C
1,32,PortC,AUD,20210715,FTC,261361,C
2,33,PortC,AUD,20210716,FTC,73055,C
3,34,PortC,AUD,20210719,FTC,4161646,C
4,35,PortC,AUD,20210720,FTC,787878787,C
5,36,PortC,AUD,20210722,FTC,878787878,C
6,37,PortC,AUD,20210723,FTC,3349533,C
7,38,PortC,AUD,20210728,FTC,8748787,C
8,39,PortC,AUD,20210812,FTC,4864848616,C
9,40,PortC,CAD,20210713,FSC,854684848,C


## Extract a 'Virtual Document' from the Structured Results Store

In [6]:
def retrieve_virtual_document():
    portfolio_ids = df['Port'].unique()

    for portfolio_id in portfolio_ids:
        # Retrieve document from SRS
        srs_id = lm.StructuredResultDataId(
            source = "Client", 
            code = portfolio_id, 
            effective_at = effective_date,
            result_type = sample_data_result_type)
        
        result = srs_api.get_virtual_document(
            scope = scope, 
            request_body = {
                "key": srs_id
            }
        )
        
        # Convert to DataFrame
        result_dfs = []

        for item in result.values["key"].data:
            columns = item.row_data.columns
            values = [i.value for i in item.row_data.values]
    
            row_df = pd.DataFrame(values).T
            row_df.columns = columns
            
            for row in item.row_id.items():            
                row_df[row[0]] = [row[1]]

            #row_df.columns = [i[i.rfind("/")+1:] for i in df.columns]
            result_dfs.append(row_df)
    
        all_dfs = pd.concat(result_dfs)
        
        return all_dfs
        
retrieve_virtual_document()

,UnitResult/srs/Port,UnitResult/srs/Value,UnitResult/srs/Class,UnitResult/srs/Currency,UnitResult/srs/Date,UnitResult/srs/Sys
0,PortA,"18,406.00",C,USD,20210810,FTC
0,PortA,"65,454.00",C,NZD,20210713,FSC
0,PortA,"544,343.00",C,USD,20210727,FTC
0,PortA,"454,534,135,453.00",C,USD,20210716,FTC
0,PortA,"3,535,454.00",C,USD,20210729,FTC
0,PortA,"12,116.00",C,USD,20210730,FTC
0,PortA,"16,875.00",C,USD,20210714,FTC
0,PortA,"45,645.00",C,NOK,20210713,FSC
0,PortA,"54,354,564.00",C,GBP,20210812,FTC
0,PortA,"50,000,000.00",C,AUD,20210713,FSC


## Cleanup - delete the data from the Structured Results Store

In [7]:
def delete_data():
    portfolio_ids = df['Port'].unique()

    for portfolio_id in portfolio_ids:
        # Retrieve document from SRS
        srs_id = lm.StructuredResultDataId(
            source = "Client", 
            code = portfolio_id, 
            effective_at = effective_date,
            result_type = sample_data_result_type)
        
        result = srs_api.delete_structured_result_data(
            scope = scope, 
            request_body = {
                "key": srs_id
            }
        )

delete_data()